# 构建检索问答链

在 `C3 搭建数据库` 章节，我们已经介绍了如何根据自己的本地知识文档，搭建一个向量知识库。 在接下来的内容里，我们将使用搭建好的向量数据库，对 query 查询问题进行召回，并将召回结果和 query 结合起来构建 prompt，输入到大模型中进行问答。   

## 1. 加载向量数据库

首先，我们加载在前一章已经构建的向量数据库。注意，此处你需要使用和构建时相同的 Emedding。

In [6]:
import sys
sys.path.append("../C3 搭建知识库") # 将父目录放入系统路径中
# 在上面的C3知识库模块中找下面要引用的东西
# 使用智谱 Embedding API，注意，需要将上一章实现的封装代码下载到本地
from zhipuai_embedding import ZhipuAIEmbeddings

from langchain.vectorstores.chroma import Chroma

加载向量数据库，其中包含了 ../../data_base/knowledge_db 下多个文档的 Embedding

In [7]:
# 定义 Embeddings
embedding = ZhipuAIEmbeddings()

# 向量数据库持久化路径
persist_directory = '../../data_base/vector_db/chroma'

# 加载数据库
vectordb = Chroma(
    persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
    embedding_function=embedding
)
print("vectordb",vectordb)

vectordb <langchain_community.vectorstores.chroma.Chroma object at 0x7ff65d1883d0>


In [8]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：20


我们可以测试一下加载的向量数据库，使用一个问题 query 进行向量检索。如下代码会在向量数据库中根据相似性进行检索，返回前 k 个最相似的文档。

> ⚠️使用相似性搜索前，请确保你已安装了 OpenAI 开源的快速分词工具 tiktoken 包：`pip install tiktoken`

In [9]:
question = "什么是prompt engineering?"
docs = vectordb.similarity_search(question,k=4)
print(f"检索到的内容数：{len(docs)}")

检索到的内容数：4


打印一下检索到的内容

In [10]:
for i, doc in enumerate(docs):
    print(f"检索到的第{i}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")
#### 输出被截断了

检索到的第0个内容: 
 3
1.4.1
式(1.1) 和式(1.2) 的解释. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
4
第2 章模型评估与选择
5
2.1
经验误差与过拟合
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
5
2.2
评估方法
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
5
2.2.1
算法参数（超参数）与模型参数. . . . . . . . . . . . . . . . . . . . . . . . . . . . .
6
2.2.2
验证集. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
6
2.3
性能度量
-----------------------------------------------------
检索到的第1个内容: 
 39
4.5.1
图(4.10) 的解释
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
39
4.5.2
图(4.11) 的解释
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
39
第5 章神经网络
41
5.1
神经元模型. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
41
5.2
感知机与多层网络
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
41
5.2.1
式(5.1) 和式(5.2) 的推导. . . .

## 2. 创建一个 LLM

在这里，我们调用 ZhiPu 的 API 创建一个 LLM，当然你也可以使用其他 LLM 的 API 进行创建

In [11]:
ZHIPUAI_API_KEY = ""
# from langchain_openai import ChatOpenAI
from zhipuai_llm import ZhipuAILLM
# llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0)
llm = ZhipuAILLM(api_key=ZHIPUAI_API_KEY)

llm.invoke("请你自我介绍一下自己！")

'你好，我是一个人工智能助手，基于清华大学 KEG 实验室和智谱 AI 公司于 2023 年共同训练的语言模型 GLM2 开发而成。我的任务是针对用户的问题和要求提供适当的答复和支持。我可以通过分析大量的文本数据来学习和理解语言，从而回答各种问题，提供信息和建议。我将继续不断学习和进化，为用户提供更好的服务。'

## 3. 构建检索问答链

In [12]:
from langchain.prompts import PromptTemplate

template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],template=template)
print(QA_CHAIN_PROMPT)
### 弄明白怎么构建的检索问答链？在代码中是怎么实现的？  这里QA_CHAIN_PROMPT是PromptTemplate的一个实例

input_variables=['context', 'question'] template='使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。\n{context}\n问题: {question}\n'


再创建一个基于模板的检索链：

In [1]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

NameError: name 'llm' is not defined

创建检索 QA 链的方法 RetrievalQA.from_chain_type() 有如下参数：
- llm：指定使用的 LLM
- 指定 chain type : RetrievalQA.from_chain_type(chain_type="map_reduce")，也可以利用load_qa_chain()方法指定chain type。
- 自定义 prompt ：通过在RetrievalQA.from_chain_type()方法中，指定chain_type_kwargs参数，而该参数：chain_type_kwargs = {"prompt": PROMPT}
- 返回源文档：通过RetrievalQA.from_chain_type()方法中指定：return_source_documents=True参数；也可以使用RetrievalQAWithSourceChain()方法，返回源文档的引用（坐标或者叫主键、索引）

## 4.检索问答链效果测试

In [15]:
question_1 = "什么是南瓜书？"
question_2 = "王阳明是谁？"

### 4.1 基于召回结果和 query 结合起来构建的 prompt 效果

In [16]:
result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
# print(result["result"])
print(result)

/root/miniconda3/envs/llm-universe/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


ValueError: `run` not supported when there is not exactly one output key. Got ['result', 'source_documents'].

In [15]:
result = qa_chain({"query": question_2})
print("大模型+知识库后回答 question_2 的结果：")
print(result["result"])

大模型+知识库后回答 question_2 的结果：
我不知道王阳明是谁，这个问题与提供的上下文无关。谢谢你的提问！


### 4.2 大模型自己回答的效果

In [16]:
prompt_template = """请回答下列问题:
                            {}""".format(question_1)
### 基于大模型的问答
llm.predict(prompt_template)

/root/miniconda3/envs/llm-universe/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'南瓜书是对周志华教授所著的《机器学习》（因其内容全面、深入，被广大读者亲切地称为“西瓜书”）中涉及的公式和数学推导进行详细解析的辅助教材。它由开源组织Datawhale发起，团队成员针对西瓜书中较为难以理解或推导的公式进行了详尽的解析和补充。\n\n南瓜书的特点在于它几乎覆盖了西瓜书中所有的重难点公式，为读者提供了必要的数学推导过程和先验知识，使得读者不仅能够知其然，还能知其所以然。它旨在帮助数学基础不牢固的读者克服学习机器学习过程中的数学难题，是西瓜书很好的补充和伴侣。\n\n自2019年发布以来，南瓜书因其内容实用、讲解清晰，受到了广大学习者的喜爱和好评，并在GitHub上获得了很高的关注度，star数量过万。业内专家也对其给予了高度评价，并推荐给有志于深入理解机器学习数学原理的学习者。南瓜书以开源形式发布，可以在线阅读和下载PDF，方便读者学习和参考。'

In [17]:
prompt_template = """请回答下列问题:
                            {}""".format(question_2)
### 基于大模型的问答
llm.predict(prompt_template)

'王阳明，原名王守仁，是明代著名的哲学家、军事家、教育家和文化名人，生活在公元1472年至1529年。他不仅在中国哲学史上占有重要地位，而且对后世有着深远的影响。王阳明以其心学哲学而著称，心学强调“知行合一”的理论，即知识和行动不可分割，主张内心的认识和实际行动应当统一。他在平定宁王叛乱等军事行动中展现出了卓越的军事才能和智慧，因此被后人尊称为“千古完人”。王阳明的学说和思想至今仍受到人们的推崇和研究，影响遍及东亚地区。'

> ⭐ 通过以上两个问题，我们发现 LLM 对于一些近几年的知识以及非常识性的专业问题，回答的并不是很好。而加上我们的本地知识，就可以帮助 LLM 做出更好的回答。另外，也有助于缓解大模型的“幻觉”问题。

## 5. 添加历史对话的记忆功能

现在我们已经实现了通过上传本地知识文档，然后将他们保存到向量知识库，通过将查询问题与向量知识库的召回结果进行结合输入到 LLM 中，我们就得到了一个相比于直接让 LLM 回答要好得多的结果。在与语言模型交互时，你可能已经注意到一个关键问题 - **它们并不记得你之前的交流内容**。这在我们构建一些应用程序（如聊天机器人）的时候，带来了很大的挑战，使得对话似乎缺乏真正的连续性。这个问题该如何解决呢？


## 1. 记忆（Memory）

在本节中我们将介绍 LangChain 中的储存模块，即如何将先前的对话嵌入到语言模型中的，使其具有连续对话的能力。我们将使用 `ConversationBufferMemory` ，它保存聊天消息历史记录的列表，这些历史记录将在回答问题时与问题一起传递给聊天机器人，从而将它们添加到上下文中。

In [18]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
    return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
)

关于更多的 Memory 的使用，包括保留指定对话轮数、保存指定 token 数量、保存历史对话的总结摘要等内容，请参考 langchain 的 Memory 部分的相关文档。

## 2. 对话检索链（ConversationalRetrievalChain）

对话检索链（ConversationalRetrievalChain）在检索 QA 链的基础上，增加了处理对话历史的能力。

它的工作流程是:
1. 将之前的对话与新问题合并生成一个完整的查询语句。
2. 在向量数据库中搜索该查询的相关文档。
3. 获取结果后,存储所有答案到对话记忆区。
4. 用户可在 UI 中查看完整的对话流程。

![](../../figures/Modular_components.png)

这种链式方式将新问题放在之前对话的语境中进行检索，可以处理依赖历史信息的查询。并保留所有信
息在对话记忆中，方便追踪。

接下来让我们可以测试这个对话检索链的效果：

使用上一节中的向量数据库和 LLM ！首先提出一个无历史对话的问题“这门课会学习到关于提示工程的知识吗？”，并查看回答。

In [19]:
from langchain.chains import ConversationalRetrievalChain

retriever=vectordb.as_retriever()

qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)
question = "我可以学习到关于提示工程的知识吗？"
result = qa({"question": question})
print(result['answer'])

不可以。根据提供的上下文信息，这里的内容主要与机器学习相关，特别是针对周志华教授的《机器学习》（西瓜书）的公式解析和推导细节。提示工程（Prompt Engineering）是自然语言处理（NLP）和人工智能（AI）领域的一个子领域，涉及设计有效的提示或指令来优化AI模型的输出。这个主题在所提供的上下文中没有涉及，因此无法通过这些材料学习到关于提示工程的知识。


然后基于答案进行下一个问题“为什么这门课需要教这方面的知识？”：

In [20]:
question = "为什么这门课需要教这方面的知识？"
result = qa({"question": question})
print(result['answer'])

《机器学习》（西瓜书）课程需要包含公式解析和推导细节的教学，是因为公式和推导是机器学习领域的核心，它们不仅能够帮助学生深入理解算法的原理和内在机制，而且对于实际应用中算法的调优和改进也是至关重要的。此外，不同学生的学习背景和对数学的理解程度不同，详细解析公式和推导过程可以帮助数学基础薄弱的学生更好地掌握机器学习的理论知识，使得他们能够“脑补”或自行推导出省略的细节，从而提高整体的学术水平和应用能力。同时，这对于那些想要深入探究机器学习理论的学生来说，也是必不可少的环节。通过公式解析和推导的教学，学生可以成长为具有扎实数学基础的机器学习专业人才。


可以看到，LLM 它准确地判断了这方面的知识，指代内容是强化学习的知识，也就
是我们成功地传递给了它历史信息。这种持续学习和关联前后问题的能力，可大大增强问答系统的连续
性和智能水平。